# 13.1

In [34]:
import pandas as pd

nhefs = pd.read_csv('../data/nhefs.csv')
nhefs.describe()

,seqn,qsmk,death,yrdth,modth,dadth,sbp,dbp,sex,age,...,birthcontrol,pregnancies,cholesterol,hightax82,price71,price82,tax71,tax82,price71_82,tax71_82
count,1629.000000,1629.000000,1629.000000,318.000000,322.000000,322.000000,1552.000000,1548.000000,1629.000000,1629.000000,...,1629.000000,726.00000,1613.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000
mean,16552.364641,0.262738,0.195212,87.569182,6.257764,15.872671,128.709407,77.744832,0.509515,43.915285,...,1.084715,3.69146,219.973962,0.165908,2.138750,1.806095,1.058581,0.505983,0.332741,0.552614
std,7498.918195,0.440256,0.396485,2.659415,3.615304,8.905488,19.051560,10.634864,0.500063,12.170430,...,0.947747,2.20560,45.444202,0.372119,0.229016,0.130641,0.216229,0.111894,0.155045,0.150321
min,233.000000,0.000000,0.000000,83.000000,1.000000,1.000000,87.000000,47.000000,0.000000,25.000000,...,0.000000,1.00000,78.000000,0.000000,1.506592,1.451904,0.524902,0.219971,-0.202698,0.035995
25%,10607.000000,0.000000,0.000000,85.000000,3.000000,8.000000,116.000000,70.000000,0.000000,33.000000,...,0.000000,2.00000,189.000000,0.000000,2.036621,1.739990,0.944946,0.439941,0.200989,0.460999
50%,20333.000000,0.000000,0.000000,88.000000,6.000000,15.500000,126.000000,77.000000,1.000000,44.000000,...,1.000000,3.00000,216.000000,0.000000,2.167969,1.814941,1.049805,0.505981,0.335999,0.543945
75%,22719.000000,1.000000,0.000000,90.000000,10.000000,24.000000,140.000000,85.000000,1.000000,53.000000,...,2.000000,5.00000,245.000000,0.000000,2.241699,1.867676,1.154785,0.571899,0.443787,0.621948
max,25061.000000,1.000000,1.000000,92.000000,12.000000,31.000000,229.000000,130.000000,1.000000,74.000000,...,2.000000,15.00000,416.000000,1.000000,2.692871,2.103027,1.522461,0.747925,0.612061,0.884399


In [35]:
nhefs['cens'] = nhefs.wt82.isnull()

In [52]:
covars = ['sex', 'race', 'age', 'education', 'smokeintensity', 'smokeyrs', 'exercise', 'active', 'wt71', 'wt82', 'qsmk', 'wt82_71']

res_nhefs = nhefs[covars]

In [53]:
res_nhefs = res_nhefs.dropna()

In [54]:
res_nhefs.count()

sex               1566
race              1566
age               1566
education         1566
smokeintensity    1566
smokeyrs          1566
exercise          1566
active            1566
wt71              1566
wt82              1566
qsmk              1566
wt82_71           1566
dtype: int64

In [55]:
import statsmodels.formula.api as smf

In [66]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))
model.append(LookupFactor('qsmk'))
model.append(EvalFactor('qsmk:smokeintensity'))


m = ModelDesc([LookupFactor('wt82_71')], model)
m.describe()

'wt82_71 ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2) + qsmk + qsmk: smokeintensity'

In [67]:
fit_model = smf.OLS.from_formula(m.describe(), res_nhefs).fit()

In [68]:
predicted_mean = fit_model.predict()

In [69]:
print(fit_model.summary())

                            OLS Regression Results                            
Dep. Variable:                wt82_71   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     13.45
Date:                Sun, 28 Jan 2018   Prob (F-statistic):           1.47e-41
Time:                        15:28:22   Log-Likelihood:                -5328.6
No. Observations:                1566   AIC:                         1.070e+04
Df Residuals:                    1545   BIC:                         1.081e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [70]:
nhefs[nhefs.seqn == 24770][covars]

,sex,race,age,education,smokeintensity,smokeyrs,exercise,active,wt71,wt82,qsmk,wt82_71
1581,0,0,26,4,15,12,1,0,111.58,114.75887,0,3.17887


In [76]:
print nhefs[nhefs.seqn == 24770]['wt82_71']

1581    3.17887
Name: wt82_71, dtype: float64


In [75]:
covars = ['sex', 'race', 'age', 'education', 'smokeintensity', 'smokeyrs', 'exercise', 'active', 'wt71', 'wt82', 'qsmk']


fit_model.predict(nhefs[nhefs.seqn == 24770][covars])

1581    0.342157
dtype: float64

In [77]:
predicted_mean.mean()

2.6382997865633

In [78]:
res_nhefs.wt82_71.mean()

2.6382997865644957

# 13.2

In [80]:
data = {
'd':["Rheia", "Kronos", "Demeter", "Hades", "Hestia", "Poseidon", 
  "Hera", "Zeus", "Artemis", "Apollo", "Leto", "Ares", "Athena", 
  "Hephaestus", "Aphrodite", "Cyclope", "Persephone", "Hermes", 
  "Hebe", "Dionysus"], 
'L':[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
'A':[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
'Y':[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
}

In [81]:
gods = pd.DataFrame(data)

In [89]:
observed = gods.copy()
observed['interv'] = -1

In [91]:
untreated = gods.copy()
untreated['Y'] = pd.np.nan
untreated['A'] = 0
untreated['interv'] = 0

In [92]:
treated = gods.copy()
treated['Y'] = pd.np.nan
treated['A'] = 1
treated['interv'] = 1

In [95]:
data = pd.concat([observed, untreated, treated])

In [83]:
fit_model = smf.Logit.from_formula('Y ~ A*L', gods).fit()

Optimization terminated successfully.
         Current function value: 0.606843
         Iterations 5


In [86]:
print fit_model.summary2()

                        Results: Logit
Model:              Logit            No. Iterations:   5.0000 
Dependent Variable: Y                Pseudo R-squared: 0.125  
Date:               2018-01-28 15:37 AIC:              32.2737
No. Observations:   20               BIC:              36.2566
Df Model:           3                Log-Likelihood:   -12.137
Df Residuals:       16               LL-Null:          -13.863
Converged:          1.0000           Scale:            1.0000 
---------------------------------------------------------------
             Coef.   Std.Err.     z     P>|z|    [0.025  0.975]
---------------------------------------------------------------
Intercept   -1.0986    1.1547  -0.9514  0.3414  -3.3618  1.1646
A           -0.0000    1.6330  -0.0000  1.0000  -3.2006  3.2006
L            1.7918    1.6833   1.0645  0.2871  -1.5074  5.0909
A:L          0.0000    2.1602   0.0000  1.0000  -4.2340  4.2340



In [87]:
fit_model.predict()

array([0.25      , 0.25      , 0.25      , 0.25      , 0.25      ,
       0.25      , 0.25      , 0.25      , 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667])

In [98]:
data['preds'] = fit_model.predict(data)

In [99]:
pd.pivot_table(data, ['preds'], columns=['interv'])

interv,-1,0,1
preds,0.5,0.5,0.5


# 13.3

In [100]:
res_nhefs.describe()

,sex,race,age,education,smokeintensity,smokeyrs,exercise,active,wt71,wt82,qsmk,wt82_71
count,1566.00000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000
mean,0.51341,0.131545,43.659642,2.714559,20.525543,24.588123,1.194764,0.646871,70.830920,73.469219,0.257344,2.638300
std,0.49998,0.338104,11.989814,1.187100,11.771588,12.008606,0.735063,0.647295,15.314903,16.158047,0.437310,7.879913
min,0.00000,0.000000,25.000000,1.000000,1.000000,1.000000,0.000000,0.000000,39.580000,35.380205,0.000000,-41.280470
25%,0.00000,0.000000,33.000000,2.000000,10.000000,15.000000,1.000000,0.000000,59.530000,61.688562,0.000000,-1.478399
50%,1.00000,0.000000,43.000000,3.000000,20.000000,24.000000,1.000000,1.000000,69.230000,72.121187,0.000000,2.603811
75%,1.00000,0.000000,53.000000,3.000000,30.000000,33.000000,2.000000,1.000000,79.802500,83.460996,1.000000,6.689581
max,1.00000,1.000000,74.000000,5.000000,80.000000,64.000000,2.000000,2.000000,151.730000,136.531303,1.000000,48.538386


In [101]:
observed = res_nhefs.copy()
observed['interv'] = -1

In [102]:
untreated = res_nhefs.copy()
untreated['wt82_71'] = pd.np.nan
untreated['qsmk'] = 0
untreated['interv'] = 0

In [103]:
treated = res_nhefs.copy()
treated['wt82_71'] = pd.np.nan
treated['qsmk'] = 1
treated['interv'] = 1

In [104]:
onesample = pd.concat([observed, untreated, treated])

In [106]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))
model.append(LookupFactor('qsmk'))


m = ModelDesc([LookupFactor('wt82_71')], model)
m.describe()

'wt82_71 ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2) + qsmk'

In [107]:
fit_model = smf.OLS.from_formula(m.describe(), onesample).fit()

In [108]:
print(fit_model.summary())

                            OLS Regression Results                            
Dep. Variable:                wt82_71   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     14.06
Date:                Sun, 28 Jan 2018   Prob (F-statistic):           8.77e-42
Time:                        15:50:44   Log-Likelihood:                -5329.5
No. Observations:                1566   AIC:                         1.070e+04
Df Residuals:                    1546   BIC:                         1.081e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [109]:
onesample['preds'] = fit_model.predict(onesample)

In [111]:
pd.pivot_table(onesample, ['preds'], columns=['interv'])

interv,-1,0,1
preds,2.6383,1.747216,5.209838


In [112]:
onesample.head()

,sex,race,age,education,smokeintensity,smokeyrs,exercise,active,wt71,wt82,qsmk,wt82_71,interv,preds
0,0,1,42,1,30,29,2,0,79.04,68.946040,0,-10.093960,-1,4.271756
1,0,0,36,2,20,24,0,0,58.63,61.234970,0,2.604970,-1,6.337239
2,1,1,56,2,20,26,2,0,56.81,66.224486,0,9.414486,-1,1.989807
3,0,1,68,1,3,53,2,1,59.42,64.410117,0,4.990117,-1,-4.261624
4,0,0,40,2,20,19,1,1,87.09,92.079251,0,4.989251,-1,2.290883


# 13.4

In [132]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))
model.append(LookupFactor('qsmk'))


m = ModelDesc([LookupFactor('wt82_71')], model)
m.describe()

def g_formula(ids):
    df = res_nhefs.iloc[ids].copy()
    
    observed = df.copy()
    observed['interv'] = -1
    
    untreated = res_nhefs.copy()
    untreated['wt82_71'] = pd.np.nan
    untreated['qsmk'] = 0
    untreated['interv'] = 0
    
    treated = res_nhefs.copy()
    treated['wt82_71'] = pd.np.nan
    treated['qsmk'] = 1
    treated['interv'] = 1
    
    fit_model = smf.OLS.from_formula(m.describe(), observed).fit()
    
    untreated_mean = fit_model.predict(untreated).mean()
    treated_mean = fit_model.predict(treated).mean()
    
    return untreated_mean - treated_mean


In [133]:
import seaborn as sns

In [134]:
bs = sns.algorithms.bootstrap(range(len(res_nhefs)), n_boot=100, func=g_formula)

In [138]:
bs.sort()

In [141]:
bs[5], bs[95]

(-4.2928977510341815, -2.802424490773374)